In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install -U g4f
!pip install tiktoken==0.6.0

In [2]:
from trl import SFTTrainer
from g4f.client import Client
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import FastLanguageModel
import torch
import pandas as pd
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth import PatchDPOTrainer
from transformers import TrainingArguments
from trl import DPOTrainer
from unsloth import is_bfloat16_supported
from typing import List
import os, json
import tiktoken
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth


In [4]:

class OpenAILLM:
    def __init__(self):
        self.model = "gpt-3.5-turbo"

    def __call__(self, prompt):
     client = Client()
     print("prompt :%%%%%%%%%%%%%", prompt)
     completion = client.chat.completions.create(
         model=self.model,
         messages=[{"role": "user", "content": prompt}],
     )

     response = completion.choices[0].message.content
     print("response :$$$$$$$$$$$$$$$$$$$$$$", response)
     return response

In [14]:
SUMMARIZE_INSTRUCTION = """Given a list of tweets, summarize all key facts in **English** regarding {ticker} stock.
Here are some examples:
{examples}
(END OF EXAMPLES)

Tweets:
{tweets}

Facts:"""
SUMMARIZE_EXAMPLES = """Tweets:
RT @ValaAfshar: Apple has $231.5 billion in cash. It could buy:
Uber
Tesla
Twitter
Airbnb
Netflix
Snapchat
SpaceX
and still have $21 bi…
#Apple announces Q3 2016 revenue of $42.4b: 40.4m iPhones, 9.9m iPads, 4.2m Macs. Read more: https://t.co/wBrFWdTL0M $AAPL
$AAPL Apple China Sales Down ~29% Sequentially, Down 33% YoY
RT @VisualStockRSRC: $AAPL - Apple Earnings Fall on IPhone Slump -- 3rd Update https://t.co/5pSSxlGq8z
RT @DanielEran: Wow: 3/4 contactless payments in US are done via Apple Pay $AAPL
RT @appleinsider: More expensive #iPadPro helps boost $AAPL Q3 revenues despite lower unit sales https://t.co/ZoOi8v93cw https://t.co/AMeR6…
#Apple puts Bob Mansfield in charge of Project Titan car project. Read more: https://t.co/SfPmr2TvPb $AAPL
Apple's market weight rating reiterated at Wells Fargo  &amp; Co.. $120.00 PT. https://t.co/hhQe9eXC1j $AAPL #AAPL
As iPhone sale sag, #Apple touts apps and services instead. Read more: https://t.co/mQK4iZEgCh $AAPL
#Apple granted patent to allow Apple Pencil to work with a Mac's trackpad. Read more: https://t.co/caQmIjfb1L $AAPL
RT @RogerWCheng: Apple @tim_cook says the services business (App Store, Apple Music, etc) will be the size of a Fortune 500 company next ye…
RT @kyurieff: Apple stock soars in after-hours on Q3 beat, pops 5%: https://t.co/MYRmtUIE4J via @TheStreet $AAPL

Facts:
Apple reported Q3 2016 earnings: Revenue of $42.4 billion, beating expectations. They sold 40.4 million iPhones, 9.9 million iPads, and 4.2 million Macs during that quarter.
Apple's earnings beat expectations, causing the stock to rise by almost 5% in after-hours trading.
Apple had $231.5 billion in cash reserves, enough to potentially acquire companies like Uber, Tesla, Twitter, Airbnb, Netflix, Snapchat, and SpaceX and still have billions left.
Apple's China sales were down around 29% sequentially and 33% YoY.
Despite declining unit sales, Apple's revenue was boosted by more expensive iPad Pro models.
Apple Pay accounted for 3/4 of contactless payments in the US.
Apple's services business (App Store, Apple Music, etc.) was projected to be the size of a Fortune 500 company in the next year.
Apple was reported to be working on a car project called Project Titan, with Bob Mansfield leading it.
The Apple Pencil was granted a patent to work with a Mac's trackpad.
Apple faced declining iPhone sales, but the company focused on promoting apps and services.
The stock price experienced fluctuations after the earnings report, with significant after-hours gains.
Apple's market weight rating was reiterated by Wells Fargo, with a target price of $120.00.

Tweets:
Apple $AAPL revenue down 13%, its first decline in 13 years.
RT @vindugoel: Apple revenue drops 13 percent, ending 13 years of growth. Greater China was especially weak, down 26 percent. $AAPL https:/…
Apple Reports Q2 EPS $1.90 vs. Est. $2.00, Rev. $50.56B vs. Est. $51.97b $AAPL https://t.co/oazu0hpNce
$AAPL Q3 guide: Sales $41B-$43B (vs $47.4B est), GM 37-5%-38% (vs 39.2%) #tech #iPhone #apple
RT @techledes: CEO Tim Cook says it was a "challenging quarter" for $AAPL, which faced "strong macroeconomic headwinds." iPhone, iPad, Macs…
$AAPL #Apple misses on profit and revenue, plans to raise dividend, return $50B more to shareholders https://t.co/AwNq1GY8yr ups #dividend
RT @IGSquawk: $AAPL Apple (Q2 16): Adj EPS $1.90 (est $2.00): Revenue $50.56 bn (est $52.00bn)
Stock down 4.8% JG
RT @USATODAYmoney: Ouch! Apple's earnings of $1.90 a share were well below the $2 the Street expected https://t.co/o6YWFIl4UK $AAPL
RT @usatodaytech: Apple reports first quarterly iPhone sales drop since 2007 debut $AAPL https://t.co/aIuozzuhiP
Apple dividend yield up to 2.3% now. $AAPL

Facts:
Apple reported its Q2 2016 earnings, missing both profit and revenue estimates.
Apple's revenue for the quarter was $50.56 billion, falling short of the estimated $52 billion.
The company's adjusted earnings per share (EPS) was $1.90, lower than the expected $2.00.
This marks the first time in 13 years that Apple experienced a quarterly decline in revenue.
iPhone sales experienced a decline for the first time since its debut in 2007.
The company's guidance for the next quarter indicates expected sales of $41 billion to $43 billion.
Apple's dividend yield increased to 2.3%.
CEO Tim Cook attributed the challenges to strong macroeconomic headwinds, especially in China.
Despite the earnings miss, Apple announced plans to raise its dividend and return $50 billion more to shareholders.
Apple's stock price experienced a decline of around 4.8% in after-hours trading following the earnings report.
"""
""
PREDICT_INSTRUCTION = """Given a list of facts, estimate their overall impact on the price movement of {ticker} stock. Give your response in this format:
(1) Price Movement, which should be either Positive or Negative.
(2) Explanation, which should be in a single, short paragraph.
Here are some examples:
{examples}
(END OF EXAMPLES)

Facts:
{summary}

Price Movement:"""
PREDICT_EXAMPLES = """Facts:
2016-07-26
Apple reported Q3 2016 earnings: Revenue of $42.4 billion, beating expectations. They sold 40.4 million iPhones, 9.9 million iPads, and 4.2 million Macs during that quarter.
Apple's earnings beat expectations, causing the stock to rise by almost 5% in after-hours trading.
Apple had $231.5 billion in cash reserves, enough to potentially acquire companies like Uber, Tesla, Twitter, Airbnb, Netflix, Snapchat, and SpaceX and still have billions left.
Apple's China sales were down around 29% sequentially and 33% YoY.
Despite declining unit sales, Apple's revenue was boosted by more expensive iPad Pro models.
Apple Pay accounted for 3/4 of contactless payments in the US.
Apple's services business (App Store, Apple Music, etc.) was projected to be the size of a Fortune 500 company in the next year.
Apple was reported to be working on a car project called Project Titan, with Bob Mansfield leading it.
The Apple Pencil was granted a patent to work with a Mac's trackpad.
Apple faced declining iPhone sales, but the company focused on promoting apps and services.
The stock price experienced fluctuations after the earnings report, with significant after-hours gains.
Apple's market weight rating was reiterated by Wells Fargo, with a target price of $120.00.

Price Movement: Positive

Explanation: Apple reported strong Q3 2016 earnings, surpassing revenue expectations and delivering robust sales figures across its product lines, including iPhones, iPads, and Macs. This performance exceeded market projections and triggered a nearly 5% increase in the stock's after-hours trading. Additionally, Apple's substantial cash reserves of $231.5 billion, capable of facilitating major acquisitions, demonstrated the company's financial stability and growth potential. Despite challenges in China, Apple's diverse revenue sources, including higher-priced iPad Pro models and the dominant Apple Pay in US contactless payments, contributed positively to its overall Price Movement. The promising growth trajectory of Apple's services business added further optimism. While facing declining iPhone sales, Apple's strategic focus on promoting apps and services reflected adaptability in response to changing market dynamics. The consistent support from Wells Fargo with a reiterated market weight rating and target price also reinforced investor confidence. The stock's fluctuations were notable but aligned with the positive earnings report, showcasing the market's responsiveness to Apple's performance.

Facts:
2016-04-26
Apple reported its Q2 2016 earnings, missing both profit and revenue estimates.
Apple's revenue for the quarter was $50.56 billion, falling short of the estimated $52 billion.
The company's adjusted earnings per share (EPS) was $1.90, lower than the expected $2.00.
This marks the first time in 13 years that Apple experienced a quarterly decline in revenue.
iPhone sales experienced a decline for the first time since its debut in 2007.
The company's guidance for the next quarter indicates expected sales of $41 billion to $43 billion.
Apple's dividend yield increased to 2.3%.
CEO Tim Cook attributed the challenges to strong macroeconomic headwinds, especially in China.
Despite the earnings miss, Apple announced plans to raise its dividend and return $50 billion more to shareholders.
Apple's stock price experienced a decline of around 4.8% in after-hours trading following the earnings report.

Price Movement: Negative

Explanation: Apple reported disappointing Q2 2016 earnings, missing both profit and revenue estimates. The company's revenue and adjusted earnings per share fell short of expectations, marking the first quarterly revenue decline in 13 years. iPhone sales, a cornerstone of Apple's business, experienced their first-ever decline since the product's debut in 2007. The weaker-than-expected guidance for the next quarter further dampened investor Price Movement. The CEO's acknowledgment of strong macroeconomic headwinds, particularly in China, indicated external challenges affecting the company's performance. Despite announcing plans to increase dividends and return more to shareholders, the stock price plunged around 4.8% in after-hours trading following the earnings report. Overall, these factors collectively indicate a negative Price Movement surrounding AAPL stock due to its underwhelming financial performance and market outlook.
"""
REFLECTION_HEADER = 'You have attempted to tackle the following task before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly tackling the given task.\n'
PREDICT_REFLECT_INSTRUCTION = """Given a list of facts, estimate their overall impact on the price movement of {ticker} stock. Give your response in this format:
(1) Price Movement, which should be either Positive or Negative.
(2) Explanation, which should be in a single, short paragraph.
Here are some examples:
{examples}
(END OF EXAMPLES)

{reflections}

Facts:
{summary}

Price Movement:"""

REFLECT_INSTRUCTION = """You are an advanced reasoning agent that can improve based on self refection. You will be given a previous reasoning trial in which you were given access to a list of facts to assess their overall impact on the price movement of {ticker} stock. You were unsuccessful in tackling the task because you gave the wrong price movement. In a few sentences, Diagnose a possible reason for failure and devise a new, concise, high level plan that aims to mitigate the same failure. Use complete sentences and use english language.

Previous trial:
{scratchpad}

Reflection:"""
class Summarizer:
    def __init__(self):
        self.summarize_prompt = SUMMARIZE_INSTRUCTION
        self.summarize_examples = SUMMARIZE_EXAMPLES
        self.llm = OpenAILLM()
        self.enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

    def get_summary(self, ticker, tweets):
        summary = None
        if tweets != []:
            prompt = self.summarize_prompt.format(
                                    ticker = ticker,
                                    examples = self.summarize_examples,
                                    tweets = "\n".join(tweets))

            while len(self.enc.encode(prompt)) > 16385:
                tweets = tweets[:-1]
                prompt = self.summarize_prompt.format(
                                        ticker = ticker,
                                        examples = self.summarize_examples,
                                        tweets = "\n".join(tweets))

            summary = self.llm(prompt)

        return summary


    def is_informative(self, summary):
         neg = r'.*[nN]o.*information.*|.*[nN]o.*facts.*|.*[nN]o.*mention.*|.*[nN]o.*tweets.*|.*do not contain.*'
         return not re.match(neg, summary)


In [ ]:
!git clone --depth 1 --filter=blob:none --sparse https://github.com/koa-fin/sep.git
%cd sep
!git sparse-checkout set data

Cloning into 'sep'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 16 (delta 0), reused 15 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 8.81 KiB | 1002.00 KiB/s, done.
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 2 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 2.63 KiB | 2.63 MiB/s, done.
/content/sep
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 312 (delta 25), reused 312 (delta 25), pack-reused 0
Receiving objects: 100% (312/312), 1.44 MiB | 8.83 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [6]:



class DataLoader:
    def __init__(self):

        self.price_dir = "/content/sep/data/sample_price/preprocessed/"
        self.tweet_dir = "/content/sep/data/sample_tweet/raw/"
        self.seq_len = 5
        self.summarizer = Summarizer()


    def daterange(self, start_date, end_date):
        for n in range(int((end_date - start_date).days)):
            yield start_date + timedelta(n)


    def get_sentiment(self, date_str, price_path):
        price_data = np.genfromtxt(price_path, dtype=str, skip_header=False)

        print(price_data[price_data[:, 0] == date_str][0, 1])
        price_chg = price_data[price_data[:, 0] == date_str][0, 1].astype(float)

        if price_chg > 0.0:
            sentiment = "Positive"
        else:
            sentiment = "Negative"
        return sentiment


    def get_tweets(self, ticker, date_str):
        tweets = []
        tweet_path = os.path.join(self.tweet_dir, ticker, date_str)

        if os.path.exists(tweet_path):
            with open(tweet_path) as f:
                lines = f.readlines()
                for line in lines:
                    tweet_obj = json.loads(line)
                    tweets.append(tweet_obj['text'])
        return tweets


    def load(self, flag):
        data = pd.DataFrame()

        for file in os.listdir(self.price_dir):
            price_path = os.path.join(self.price_dir, file)
            ordered_price_data = np.flip(np.genfromtxt(price_path, dtype=str, skip_header=False), 0)
            ticker = file[:-4]

            tes_idx = round(len(ordered_price_data) * 0.8)
            end_idx = len(ordered_price_data)

            if flag == "train":
                data_range = range(tes_idx)
            else:
                data_range = range(tes_idx, end_idx)
                print("data_range",data_range)

            for idx in data_range:
                summary_all = ""

                end_date_str = ordered_price_data[idx, 0]
                end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
                start_date = end_date - timedelta(days=self.seq_len)
                target = self.get_sentiment(end_date_str, price_path)

                for seq_date in self.daterange(start_date, end_date):
                    seq_date_str = seq_date.strftime("%Y-%m-%d")

                    tweet_data = self.get_tweets(ticker, seq_date_str)
                    summary = self.summarizer.get_summary(ticker, tweet_data)

                    if summary and summary is not None and summary != "" and self.summarizer.is_informative(summary):
                        summary_all = summary_all + seq_date_str + "\n" + summary + "\n\n"

                if summary_all != "":
                     row = pd.DataFrame([{'ticker': ticker, 'summary': summary_all.rstrip(), 'target': target}])
                     data = pd.concat([data, row], ignore_index=True)
                     data.to_csv("data.csv", index=False)
        return data


In [7]:

# Use the assigned values to initialize your DataLoader
dataloader = DataLoader()
data = dataloader.load(flag="train")
# data = pd.read_csv("data-summery-gpt3-5.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/content/sep/data/sample_price/preprocessed/'

In [15]:
class NShotLLM:
    def __init__(self, model=None, tokenizer=None,):
        self.model = model
        self.tokenizer = tokenizer





    def __call__(self, prompt):

        query = self.tokenizer.encode(prompt, return_tensors="pt")
        output_ids = self.model.generate(
            query,
            do_sample=True,
            temperature=0.7,
            max_new_tokens=1024,
        )
        response = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        print(response)
        return response[0]

In [16]:
class PredictAgent:
    def __init__(self,
                 ticker: str,
                 summary: str,
                 target: str,
                 predict_llm = OpenAILLM()
                 ) -> None:

        self.ticker = ticker
        self.summary = summary
        self.target = target
        self.prediction = ''

        self.predict_prompt = PREDICT_INSTRUCTION
        self.predict_examples = PREDICT_EXAMPLES
        self.llm = predict_llm

        self.__reset_agent()

    def run(self, reset=True) -> None:
        if reset:
            self.__reset_agent()

        facts = "Facts:\n" + self.summary + "\n\nPrice Movement: "
        self.scratchpad += facts
        print(facts, end="")

        self.scratchpad += self.prompt_agent()
        response = self.scratchpad.split('Price Movement: ')[-1]
        self.prediction = response.split()[0]
        print(response, end="\n\n\n\n")

        self.finished = True

    def prompt_agent(self) -> str:
        return self.llm(self._build_agent_prompt())

    def _build_agent_prompt(self) -> str:
        return self.predict_prompt.format(
                            ticker = self.ticker,
                            examples = self.predict_examples,
                            summary = self.summary)

    def is_finished(self) -> bool:
        return self.finished

    def is_correct(self) -> bool:
        return EM(self.target, self.prediction)

    def __reset_agent(self) -> None:
        self.finished = False
        self.scratchpad: str = ''

In [17]:



class PredictReflectAgent(PredictAgent):
    def __init__(self,
                 ticker: str,
                 summary: str,
                 target: str,
                 predict_llm = OpenAILLM(),
                 reflect_llm = OpenAILLM()
                 ) -> None:

        super().__init__(ticker, summary, target, predict_llm)
        self.predict_llm = predict_llm
        self.reflect_llm = reflect_llm
        self.reflect_prompt = REFLECT_INSTRUCTION
        self.agent_prompt = PREDICT_REFLECT_INSTRUCTION
        self.reflections = []
        self.reflections_str: str = ''

    def run(self, reset=True) -> None:
        if self.is_finished() and not self.is_correct():
            self.reflect()

        PredictAgent.run(self, reset=reset)

    def reflect(self) -> None:
        print('Reflecting...\n')
        reflection = self.prompt_reflection()
        self.reflections += [reflection]
        self.reflections_str = format_reflections(self.reflections)
        print(self.reflections_str, end="\n\n\n\n")

    def prompt_reflection(self) -> str:
        return self.reflect_llm(self._build_reflection_prompt())

    def _build_reflection_prompt(self) -> str:
        return self.reflect_prompt.format(
                            ticker = self.ticker,
                            scratchpad = self.scratchpad)


    def run_n_shots(self, model, tokenizer, reset=True) -> None:
        self.llm = NShotLLM(model, tokenizer)
        PredictAgent.run(self, reset=reset)

    def _build_agent_prompt(self) -> str:
        prompt = self.agent_prompt.format(
                            ticker = self.ticker,
                            examples = self.predict_examples,
                            reflections = self.reflections_str,
                            summary = self.summary)
        return prompt




def format_reflections(reflections: List[str], header: str = REFLECTION_HEADER) -> str:
    if reflections == []:
        return ''
    else:
        return header + 'Reflections:\n- ' + '\n- '.join([r.strip() for r in reflections])

def EM(prediction, sentiment) -> bool:
    return prediction.lower() == sentiment.lower()

In [18]:
def summarize_trial(agents):
    correct = [a for a in agents if a.is_correct()]
    incorrect = [a for a in agents if a.is_finished() and not a.is_correct()]
    return correct, incorrect

In [21]:

agent_cls = PredictReflectAgent

agents = [agent_cls(row['ticker'], row['summary'], row['target']) for _, row in data.iterrows()]

print("Loaded Train Agents.")

samples = []
for agent in agents:
    agent.run()
    if agent.is_correct():
        prompt = agent._build_agent_prompt()
        response = agent.scratchpad.split('Price Movement: ')[-1]
        sample = {"instruction": prompt, "input": "", "output": response}
        samples.append(sample)

with open('merge_sample.json', 'w') as f:
    json.dump(samples, f, indent=2)



Loaded Train Agents.
Facts:
2021-12-05
Here are the key facts about BABA stock mentioned in the tweets:

* Alibaba's stock price has fallen from $320 to $110.
* The company's delisting concerns have been a major worry for investors.
* JP Morgan has valued Alibaba at 0.3x 2022E core ecommerce profit before loss from CGP in their SOTP valuation.
* Alibaba's dividend yield is not mentioned in these tweets.
* The company's revenue and earnings are not mentioned in these tweets.
* Alibaba's stock has been actively traded, with a high put/call ratio and call premium.
* The company's ADR is fully fungible into eight HK shares, providing an alternative for investors concerned about delisting.
* Alibaba's cloud business has been mentioned as a potential growth area.
* The company's valuation has been estimated by some analysts to be around $207 per share using a SOTP valuation.
* Alibaba's stock has been compared to other Chinese tech stocks, such as Didi, Baidu, and JD.com.
* The company's reg

KeyboardInterrupt: 

In [ ]:
correct, incorrect = summarize_trial(agents)
print(f'Finished Trial 0, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

Finished Trial 0, Correct: 24, Incorrect: 4


In [ ]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


dataset =load_dataset("json", data_files="/content/merge_sample.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

In [ ]:


# Split dataset into train and eval sets
# eval_size = int(0.1 * len(dataset))
# train_dataset = dataset.select(range(eval_size, len(dataset)))
# eval_dataset = dataset.select(range(eval_size))
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,

    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/24 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.594 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 24 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.729400
2,1.721500
3,1.738400
4,1.713200
5,1.736000
6,1.732800
7,1.699800
8,1.710600
9,1.726800
10,1.687400


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# fine tune with new date

In [8]:
!gdown  --folder 1-ArFyYrhpzlBNvNfwYZ_k878PFAatFHk

Retrieving folder contents
Processing file 1-Iz82xUTFkZzIY3fL3Ow73Gy1eZHoMyW adapter_config.json
Processing file 1-Ate_8gBEMg0Ey79WTatjR9LhP-2KOIn adapter_model.safetensors
Processing file 1-Em7HuqhvsNQ2hpFX5s_HEnXFapX86qD README.md
Processing file 1-I_RNJCDSAxET2Pglo6bF1MXYqouDBfO special_tokens_map.json
Processing file 1-B2tUihBxkpcV4hfETeGeXzpJnvL5kMz tokenizer_config.json
Processing file 1-I0ag10tJGplqRMf3DWtGbRdhCJuhcph tokenizer.json
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-Iz82xUTFkZzIY3fL3Ow73Gy1eZHoMyW
To: /content/lora_model/adapter_config.json
100% 732/732 [00:00<00:00, 3.37MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-Ate_8gBEMg0Ey79WTatjR9LhP-2KOIn
From (redirected): https://drive.google.com/uc?id=1-Ate_8gBEMg0Ey79WTatjR9LhP-2KOIn&confirm=t&uuid=fc440c35-4d6f-49ed-8725-17d43faf8126
To: /content/lora_model/adapter_model.safetensors
1

# fine tune with old date

In [ ]:
# !gdown  --folder 1nUOUptVmfoHCAxRmlBTKxVqltMaXWZys

Retrieving folder contents
Processing file 1-ImBbeqCpk8RDDaplEwvt-QKqnHu7Rzp adapter_config.json
Processing file 1-3gvbjiWbVR93pNksxuRXfj2LL2tTzi- adapter_model.safetensors
Processing file 1-D5pcmTe1HNOUMNupJD4zMeXQo0akz0z README.md
Processing file 1-JNSi34x9tyE9nS3Iw2fGq5WCkU2EqTW special_tokens_map.json
Processing file 1-8WFAshBRtfg4H-Fb9EHF6gTA4zos8PK tokenizer_config.json
Processing file 1-AvjAh0RUGZFiTaiADYiq8wkXTrxlakr tokenizer.json
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-ImBbeqCpk8RDDaplEwvt-QKqnHu7Rzp
To: /content/lora_model/adapter_config.json
100% 732/732 [00:00<00:00, 2.72MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-3gvbjiWbVR93pNksxuRXfj2LL2tTzi-
From (redirected): https://drive.google.com/uc?id=1-3gvbjiWbVR93pNksxuRXfj2LL2tTzi-&confirm=t&uuid=65fd11d5-7236-4187-b672-aef20e38d311
To: /content/lora_model/adapter_model.safetensors
1

In [9]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,


)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference



# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "What is a famous tall tower in Paris?", # instruction
#         "", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

In [ ]:

def remove_reflections(prompt: str) -> str:
    prefix = prompt.split('You have attempted to tackle the following task before and failed.')[0]
    suffix = prompt.split('\n\nFacts:')[-1]
    return prefix.strip('\n').strip() + '\n\nFacts' +  suffix.strip('\n').strip()

In [ ]:
        # Collect comparison data
        comparison_data = []

        for trial in range(2):
            for idx, agent in enumerate([a for a in agents if not a.is_correct()]):

                prev_response = agent.scratchpad.split('Price Movement: ')[-1]
                agent.run()

                if agent.is_correct():
                    print("-------------" ,agent._build_agent_prompt(), "\n\n\n")
                    prompt = remove_reflections(agent._build_agent_prompt())
                    response = agent.scratchpad.split('Price Movement: ')[-1]

                    sample = {"user_input": prompt, "completion_a": prev_response, "completion_b": response}
                    comparison_data.append(sample)

            correct, incorrect = summarize_trial(agents)
            print(f'Finished Trial {trial+1}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

Reflecting...

prompt :%%%%%%%%%%%%% You are an advanced reasoning agent that can improve based on self refection. You will be given a previous reasoning trial in which you were given access to a list of facts to assess their overall impact on the price movement of BABA stock. You were unsuccessful in tackling the task because you gave the wrong price movement. In a few sentences, Diagnose a possible reason for failure and devise a new, concise, high level plan that aims to mitigate the same failure. Use complete sentences and use english language.

Previous trial:
Facts:
2020-04-03
From the tweets provided, here are the key facts regarding Alibaba (BABA) stock:

1. Alibaba (BABA) is working with New York Governor Cuomo on sourcing materials from China, possibly related to the coronavirus pandemic.
2. There's no specific financial data or performance indicators mentioned in the tweets regarding Alibaba's earnings or revenue.
3. The stock is mentioned in a list of various other stocks i

In [ ]:
        os.makedirs('/content', exist_ok=True)
        comparison_data_path = os.path.join('/content', "comparison_data.json")

        if comparison_data:
            with open(comparison_data_path, 'w') as f:
                f.write(json.dumps(comparison_data))

In [ ]:
dataset =load_dataset("json", data_files="/content/comparison_data.json",)

In [ ]:

def _strip_prefix(s, pattern):
        # Use re.escape to escape any special characters in the pattern
        return re.sub(f"^{re.escape(pattern)}", "", s)

In [ ]:

def apply_chat_template(
    example,  assistant_prefix="<|assistant|>\n"
):


        if all(k in example.keys() for k in ("user_input", "completion_a","completion_b")):

            prompt_messages=[{"role": "user", "content":example["user_input"] }]
            chosen_messages=[{"role": "assistant", "content":example["completion_b"] }]

            rejected_messages = [{"role": "assistant", "content":example["completion_a"] }]
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
            example["text_prompt"] = tokenizer.apply_chat_template(
                prompt_messages, tokenize=False, add_generation_prompt=True
            )
            example["text_chosen"] = _strip_prefix(example["text_chosen"], assistant_prefix)
            example["text_rejected"] = _strip_prefix(example["text_rejected"], assistant_prefix)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `dpo` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )

        return example

dataset = dataset.map(apply_chat_template,remove_columns = list(dataset["train"].features))
dataset['train'] = dataset["train"].rename_columns(
    {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
)

In [ ]:
dataset['train'][1]

{'chosen': "<|im_start|>assistant\nPositive\n\nExplanation: Despite some caution and uncertainty surrounding BABA stock, the overall sentiment and facts suggest a positive price movement. The stock is considered solid, with potential buying opportunities at lower prices. Its presence in investment discussions, technical analysis, and trading strategies indicates a strong market interest. The company's fundamentals, including its growth drivers and strong support level around $211, contribute to its positive outlook. Although there are concerns about competition dynamics and Ant Group's IPO cancellation, the stock's inclusion in prominent indices like FANG+ and its strong ROCE performance suggest a resilient company. The trading data and options flow also indicate a high level of market activity, which can drive the stock price upward. Overall, the balanced analysis of sentiment, facts, and fundamentals points to a positive price movement for BABA stock.<|im_end|>\n",
 'rejected': "<|im

In [ ]:


PatchDPOTrainer()

In [ ]:


model.gradient_checkpointing_enable()
dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 5e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
    beta = 0.1,
    train_dataset = dataset["train"],
    # eval_dataset = raw_datasets["test"],
    tokenizer = tokenizer,
    max_length = 1024,
    max_prompt_length = 512,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


In [ ]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 3
 "-____-"     Number of trainable parameters = 41,943,040
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen


TrainOutput(global_step=3, training_loss=0.13354810575644174, metrics={'train_runtime': 44.8642, 'train_samples_per_second': 0.201, 'train_steps_per_second': 0.067, 'total_flos': 0.0, 'train_loss': 0.13354810575644174, 'epoch': 2.6666666666666665})

In [ ]:
model.save_pretrained("dpo_model") # Local saving
tokenizer.save_pretrained("dpo_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('dpo_model/tokenizer_config.json',
 'dpo_model/special_tokens_map.json',
 'dpo_model/tokenizer.json')

# new model

In [10]:
!gdown  --folder 1-L8guuOhjyQqHCRjG8KoO237QcFRlV6i

Retrieving folder contents
Processing file 1-_PY6YxWkt_04l2hZWGfqrtjIYotHbO8 adapter_config.json
Processing file 1-Lw9JzqK3C8XfW9f15XuWLAArSz5YnO9 adapter_model.safetensors
Processing file 1-VT2EcEA98U8PZGGFDoqnwBQfypeSant README.md
Processing file 1-_K3jSNnMrT2IvmCmdE6ARwp9TY1p96I special_tokens_map.json
Processing file 1-Qm-REZM1lh_U2Jh0SEhea2t-eOMTv9b tokenizer_config.json
Processing file 1-XfsIx1Fl0TpfG0pPXs-ZD0T8hF9jx8a tokenizer.json
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-_PY6YxWkt_04l2hZWGfqrtjIYotHbO8
To: /content/dpo_model/adapter_config.json
100% 732/732 [00:00<00:00, 4.44MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-Lw9JzqK3C8XfW9f15XuWLAArSz5YnO9
From (redirected): https://drive.google.com/uc?id=1-Lw9JzqK3C8XfW9f15XuWLAArSz5YnO9&confirm=t&uuid=e00da8cd-bf55-4289-93b6-685c72035d18
To: /content/dpo_model/adapter_model.safetensors
100

# old model

In [ ]:
# !gdown  --folder 1n-tgRdYtYZbG3bHuIQDQtOxuA-LkHt8G

Retrieving folder contents
Processing file 1-C3TkD1rh1kvf7l74ijnkrr6-YU9GVhi adapter_config.json
Processing file 1-3NnE7DcFMHux9Hju9vrQGEQ6yDT3VU7 adapter_model.safetensors
Processing file 1-AonTBCWm876Obqzjqssfh3VAgKPS0BX README.md
Processing file 1-EDeE63y9wxIXHkem7TuQXgJhQOgl0Cr special_tokens_map.json
Processing file 1-4swsBfkbIFfSne8Z3fmcoN2fF5-lszr tokenizer_config.json
Processing file 1-62y5QZ1_8tBfOlf_8zdRyrIA6oRjAnD tokenizer.json
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-C3TkD1rh1kvf7l74ijnkrr6-YU9GVhi
To: /content/dpo_model/adapter_config.json
100% 732/732 [00:00<00:00, 2.52MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-3NnE7DcFMHux9Hju9vrQGEQ6yDT3VU7
From (redirected): https://drive.google.com/uc?id=1-3NnE7DcFMHux9Hju9vrQGEQ6yDT3VU7&confirm=t&uuid=82936e0a-9ab5-4f1f-ba56-517585bcb544
To: /content/dpo_model/adapter_model.safetensors
100

# test with actual model

In [12]:
        print("Loading Test Agents...")
        # dataloader = DataLoader()
        # data = dataloader.load(flag="test")
        data = pd.read_csv("test.csv")

Loading Test Agents...


In [19]:

        samples = []
        agent_cls = PredictReflectAgent
        test_agents = [agent_cls(row['ticker'], row['summary'], row['target']) for _, row in data.iterrows()]
        print("Loaded Test Agents.")
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = "/content/dpo_model", # YOUR MODEL YOU USED FOR TRAINING
            max_seq_length = max_seq_length,
            dtype = dtype,
            load_in_4bit = load_in_4bit,
             )
        for agent in test_agents:
            agent.run_n_shots(
                              model=model,
                              tokenizer=tokenizer,
                              )
            if agent.is_correct():
               prompt = agent._build_agent_prompt()
               response = agent.scratchpad.split('Price Movement: ')[-1]
               sample = {"instruction": prompt, "input": "", "output": response}
               samples.append(sample)


        with open('true_dpo.json', 'w') as f:
           json.dump(samples, f, indent=2)
        correct, incorrect = summarize_trial(test_agents)
        print(f'Finished evaluation, Correct: {len(correct)}, Incorrect: {len(incorrect)}')



Loaded Test Agents.
==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Facts:
2021-12-05
Here are the key facts about BABA stock mentioned in the tweets:

* Alibaba's stock price has fallen from $320 to $110.
* The company's delisting concerns have been a major worry for investors.
* JP Morgan has valued Alibaba at 0.3x 2022E core ecommerce profit before loss from CGP in their SOTP valuation.
* Alibaba's dividend yield is not mentioned in these tweets.
* The company's revenue and earnings are not mentioned in these tweets.
* Alibaba's stock has been actively traded, with a high put/call ratio and call premium.
* The company's ADR is fully fungible into eight HK shares, providing an alternative for investors concerned about delisting.
* Alibaba's cloud business has been mentioned as a potential growth area.
* The company's valuation has been estimated by some analysts to be around $207 per share using a SOTP valuation.
* Alibaba's stock has been compared to other Chinese tech stocks, such as Didi, Baidu, and JD.com.
* The company's regulatory environment h

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["Given a list of facts, estimate their overall impact on the price movement of BABA stock. Give your response in this format:\n(1) Price Movement, which should be either Positive or Negative.\n(2) Explanation, which should be in a single, short paragraph.\nHere are some examples:\nFacts:\n2016-07-26\nApple reported Q3 2016 earnings: Revenue of $42.4 billion, beating expectations. They sold 40.4 million iPhones, 9.9 million iPads, and 4.2 million Macs during that quarter.\nApple's earnings beat expectations, causing the stock to rise by almost 5% in after-hours trading.\nApple had $231.5 billion in cash reserves, enough to potentially acquire companies like Uber, Tesla, Twitter, Airbnb, Netflix, Snapchat, and SpaceX and still have billions left.\nApple's China sales were down around 29% sequentially and 33% YoY.\nDespite declining unit sales, Apple's revenue was boosted by more expensive iPad Pro models.\nApple Pay accounted for 3/4 of contactless payments in the US.\nApple's services 

...

["Given a list of facts, estimate their overall impact on the price movement of BABA stock. Give your response in this format:\n(1) Price Movement, which should be either Positive or Negative.\n(2) Explanation, which should be in a single, short paragraph.\nHere are some examples:\nFacts:\n2016-07-26\nApple reported Q3 2016 earnings: Revenue of $42.4 billion, beating expectations. They sold 40.4 million iPhones, 9.9 million iPads, and 4.2 million Macs during that quarter.\nApple's earnings beat expectations, causing the stock to rise by almost 5% in after-hours trading.\nApple had $231.5 billion in cash reserves, enough to potentially acquire companies like Uber, Tesla, Twitter, Airbnb, Netflix, Snapchat, and SpaceX and still have billions left.\nApple's China sales were down around 29% sequentially and 33% YoY.\nDespite declining unit sales, Apple's revenue was boosted by more expensive iPad Pro models.\nApple Pay accounted for 3/4 of contactless payments in the US.\nApple's services 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Given a list of facts, estimate their overall impact on the price movement of BABA stock. Give your response in this format:\n(1) Price Movement, which should be either Positive or Negative.\n(2) Explanation, which should be in a single, short paragraph.\nHere are some examples:\nFacts:\n2016-07-26\nApple reported Q3 2016 earnings: Revenue of $42.4 billion, beating expectations. They sold 40.4 million iPhones, 9.9 million iPads, and 4.2 million Macs during that quarter.\nApple\'s earnings beat expectations, causing the stock to rise by almost 5% in after-hours trading.\nApple had $231.5 billion in cash reserves, enough to potentially acquire companies like Uber, Tesla, Twitter, Airbnb, Netflix, Snapchat, and SpaceX and still have billions left.\nApple\'s China sales were down around 29% sequentially and 33% YoY.\nDespite declining unit sales, Apple\'s revenue was boosted by more expensive iPad Pro models.\nApple Pay accounted for 3/4 of contactless payments in the US.\nApple\'s servi

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Given a list of facts, estimate their overall impact on the price movement of BABA stock. Give your response in this format:\n(1) Price Movement, which should be either Positive or Negative.\n(2) Explanation, which should be in a single, short paragraph.\nHere are some examples:\nFacts:\n2016-07-26\nApple reported Q3 2016 earnings: Revenue of $42.4 billion, beating expectations. They sold 40.4 million iPhones, 9.9 million iPads, and 4.2 million Macs during that quarter.\nApple\'s earnings beat expectations, causing the stock to rise by almost 5% in after-hours trading.\nApple had $231.5 billion in cash reserves, enough to potentially acquire companies like Uber, Tesla, Twitter, Airbnb, Netflix, Snapchat, and SpaceX and still have billions left.\nApple\'s China sales were down around 29% sequentially and 33% YoY.\nDespite declining unit sales, Apple\'s revenue was boosted by more expensive iPad Pro models.\nApple Pay accounted for 3/4 of contactless payments in the US.\nApple\'s servi

# test with gpt 3.5

In [23]:
data = pd.read_csv("test.csv")

In [28]:

agent_cls = PredictReflectAgent

agents = [agent_cls(row['ticker'], row['summary'], row['target']) for _, row in data.iterrows()]

print("test with gpt 3.5")

samples = []
for agent in agents:
    agent.run()
    if agent.is_correct():
        prompt = agent._build_agent_prompt()
        response = agent.scratchpad.split('Price Movement: ')[-1]
        sample = {"instruction": prompt, "input": "", "output": response}
        samples.append(sample)


with open('true_gpt3.5.json', 'w') as f:
    json.dump(samples, f, indent=2)
correct, incorrect = summarize_trial(agents)
print(f'Finished evaluation, Correct: {len(correct)}, Incorrect: {len(incorrect)}')





test with gpt 3.5
Facts:
2021-12-05
Here are the key facts about BABA stock mentioned in the tweets:

* Alibaba's stock price has fallen from $320 to $110.
* The company's delisting concerns have been a major worry for investors.
* JP Morgan has valued Alibaba at 0.3x 2022E core ecommerce profit before loss from CGP in their SOTP valuation.
* Alibaba's dividend yield is not mentioned in these tweets.
* The company's revenue and earnings are not mentioned in these tweets.
* Alibaba's stock has been actively traded, with a high put/call ratio and call premium.
* The company's ADR is fully fungible into eight HK shares, providing an alternative for investors concerned about delisting.
* Alibaba's cloud business has been mentioned as a potential growth area.
* The company's valuation has been estimated by some analysts to be around $207 per share using a SOTP valuation.
* Alibaba's stock has been compared to other Chinese tech stocks, such as Didi, Baidu, and JD.com.
* The company's regula

# test with llama 3 model

In [ ]:
  data = pd.read_csv("test.csv")

  samples = []
  agent_cls = PredictReflectAgent
  test_agents = [agent_cls(row['ticker'], row['summary'], row['target']) for _, row in data.iterrows()]
  print("Loaded Test Agents.")
  model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
  for agent in test_agents:
      agent.run_n_shots(
                        model=model,
                        tokenizer=tokenizer,
                        )
      if agent.is_correct():
         prompt = agent._build_agent_prompt()
         response = agent.scratchpad.split('Price Movement: ')[-1]
         sample = {"instruction": prompt, "input": "", "output": response}
         samples.append(sample)
  with open('true_dpo.json', 'w') as f:
     json.dump(samples, f, indent=2)
  correct, incorrect = summarize_trial(test_agents)
  print(f'Finished evaluation, Correct: {len(correct)}, Incorrect: {len(incorrect)}')